In [3]:
import cv2
import numpy as np
import fct
import time



count = 0
rep = 1 #negatif = inf


list_champ = []

while count != rep :
    count = count + 1


    font_initial = cv2.imread('map.png')
    img_screen_RGB = fct.sct((1585,743),(1901,1062)) #Screen
    
    img_screen_BGR = fct.process_rgb2bgr(img_screen_RGB)
    img_screen_BGR_copy = img_screen_BGR.copy()

    img_screen_hsv_RED = fct.process_red(img_screen_BGR)
    img_screen_gray = fct.process_hsv2gray(img_screen_hsv_RED)

    circles = cv2.HoughCircles(img_screen_gray, cv2.HOUGH_GRADIENT, 1, 1, param1=700, param2=10, minRadius=14, maxRadius=15)


    if np.all(circles) != None :
        circles = np.uint16(np.around(circles))

        for (xi,yi,radi) in circles[0, :] :
            cv2.rectangle(img_screen_BGR_copy, (xi-radi,yi-radi) , (xi+radi,yi+radi) , (255,0,0) , 1)
            #cv2.putText(img_screen_BGR_copy, str(radi), (xi-8, yi-16), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, 1)
            champ_detected = img_screen_BGR.copy()[yi-18:yi+18 , xi-18:xi+18]
            champ_detected_masked = fct.masked(champ_detected.copy())
            

            value_list = []
            if list_champ == [] :
                list_champ.append([champ_detected_masked]+[xi]+[yi]+[radi]+[0])
                timer_0 = time.time() #lance timer 0
            else :
                line_counter = -1
                for z in list_champ :
                    line_counter = line_counter + 1
                    value_list.append(fct.matchim(champ_detected_masked,list_champ,line_counter))
                
                max_avg_line = value_list.index(max(value_list)) #renvoi ligne de meilleur value
                if value_list[max_avg_line] > 0.525 :
                    list_champ[max_avg_line][0] = champ_detected_masked #Rewrite screen
                    list_champ[max_avg_line][1] = xi #Rewrite x
                    list_champ[max_avg_line][2] = yi #Rewrite y
                    list_champ[max_avg_line][3] = radi #Rewrite rad
                    list_champ[max_avg_line][4] = max_avg_line #Numéro fixe d'image
                    globals()['timer_%s' %max_avg_line] = time.time() #Reset timer
                elif len(list_champ) < 5 :
                    list_champ.append([champ_detected_masked]+[xi]+[yi]+[radi]+[line_counter+1])
                    globals()['timer_%s' %(line_counter+1)] = time.time() #Lance timer


    for (im,x,y,radi,num) in list_champ :
        xdelai = -18
        ydelai = -20
        xx = x + xdelai
        yy = y + ydelai
        font_initial = fct.pastemap(font_initial,im,xx,yy)

        globals()['timer_ss_%s' %num] = int(time.time() - globals()['timer_%s' %num]) #valeur print ss
        timer = globals()['timer_ss_%s' %num]
        cv2.putText(font_initial, "{}".format(timer) , (xx+11,yy), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, 1)




    result1 = np.concatenate((img_screen_BGR, img_screen_hsv_RED,img_screen_BGR_copy), axis=1)
    cv2.imshow("Map1",result1)
    cv2.imshow("Map2",img_screen_gray)
    cv2.imshow("Map3",font_initial)



    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.waitKey()
cv2.destroyAllWindows()

fix :
-the spectral filtre
-random crash
-Superposed champ : take 2 slot for 1 champ
-

add :
-Personnal input to be worldwide used
    -offset x y print map
-Circle probability (trouver vitesse de spread)

In [81]:
cv2.imshow("Map1",list_champ[0][0])

cv2.waitKey()
cv2.destroyAllWindows()


IndexError: list index out of range

In [1]:
import cv2
import numpy as np




map = cv2.imread('capture1.png')
champ = cv2.imread('imd/0.png')
x = 200
y = 200


img1 = map
img2 = champ

img_2_shape = img2.shape
roi = img1[y:y+img_2_shape[0],x:x+img_2_shape[1]]


# prout = np.zeros((32,32), np.uint8)
# prout = cv2.bitwise_not(prout)
# cv2.circle(prout,(18,18),16,1,thickness=-1)


height,width,depth = img2.shape
mask = np.zeros((height,width), np.uint8)
mask = cv2.bitwise_not(mask)
cv2.circle(mask,(18,18),16,1,thickness=-1)
mask_inv = cv2.bitwise_not(mask)


img1_bg = cv2.bitwise_and(roi,roi,mask = mask)
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
dst = cv2.add(img1_bg,img2_fg)

img1[y:y+img_2_shape[0], x:x+img_2_shape[1]] = dst




result1 = np.concatenate((mask, mask_inv), axis=1)
cv2.imshow("0",result1)

result2 = np.concatenate((roi), axis=1)
cv2.imshow("2",result2)


cv2.imshow("1",img1)


cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
champ = cv2.imread('imd/0.png')

caca = fct.masked(champ)


cv2.imshow("1",caca)


cv2.waitKey()
cv2.destroyAllWindows()